## Ch6 Deep Feedforward Networks

feedforward 네트워크를 이해하기 위한 하가지 방법은 linear model로 부터시작하여 한계점을 어떻게 극복하려고 하는지를 알아 보는 것이다. linear model은 매우 효율적이고 closed form, 혹은 convex optimization으로 풀 수 있어서 매우 효율적입니다. 하지만 model capacity가 linear function에만 제한되어져 있어 단점을 가지고 있으며 이는 두 변수사이의 관계를 표현하지 못하게 됩니다. (오직 ax1+bx2 와 같은 것들만 표현가능... x1x2 의 상호작용은 반영하지 못한다.)

linear model을 non-linear 하게 확장하기 위해서 우리가 할 수 있는 것은 x를 $\phi(x)$로 변환하여 적용하는 것입니다. 이와 동일하게 이전에서 언급된 kernel trick을 사용할 수 있습니다. 

그렇다면 $\phi$ 를 어떻게 선택해야 할까요?

1. 매우 일반적인 형태의  $\phi$를 선택하는 것입니다. infinite-dimensional  $\phi$는 RBF kernel을 사용하여 적용될 수 있습니다.  $\phi$ 가 매우 고차원이라면 train set을 학습 할 수 있는 충분한 capacity를 가질 수 있지만 test error는 매우 안좋을 것입니다. 이러한 일반적인 형태의 feature mapping은 보통은 local smoothness prior만을 반영할 수 있으며 더 어려운 문제를 풀기위한 충분한 prior를 반영하기 어렵습니다.

2. 다른 방법은  $\phi$를 엔지니어가 모델링 해주는 것입니다. 딥러닝 이전에는 이것이 좋은 성능을 보였습니다. 이 접근방법은 각 task에 대해 적절한 커널을 모델링 하기 위해 인간의 많은 노력을 필요로 하며 여러 도메인에 걸쳐서 사용되기 힘듭니다. (비전에서의 sift 와 같은 커널을 모델링 해주는 것.. sift가 근데 non-linear인가?)

3. 딥러닝에서의 전략은  $\phi$를 학습하는 것입니다. 이 접근 방법에서는 $y=f(x;\theta,w) =  \phi(x; \theta)^Tw$ 라는 model을 가집니다. 파라미터 $\theta$를 통해  $\phi$를 다양한 function 중에서 학습할수 있도록 하며 파라미터 w는  $\phi(x)$ 를 적절한 아웃풋으로 매핑시킵니다. 이는 포워트 네트워크의 경우와 일치합니다. 이 경우 hidden layer를  $\phi$로 생각 할 수 있습니다. 이 접근 방법은 convexity 를 포기하였지만 이로 인해 얻는 안좋은 점 보다 좋은 점이 더 많습니다. 이 방법에서 $\phi(x; \theta)$ 를 파라미터를 통해 representation하며 좋은 representation에 해당하는 theta를 찾기위해opimization 알고리즘을 적용합니다. 이를 통해 첫번째 방법과 같이 매우 일반적인 형태의 함수를 학습 할 수 있을 것으로 기대 할 수 있습니다. 두번째 방법의 효과도 얻을 수 있는데 $\phi(x; \theta)$ 디자인할때 인간의 지식을 넣어 일반화를 잘하도록 도울 수 있습니다.(CNN, RNN의 형태처럼) 이로인해 엔지니어들은 정확한 function을 찾는 것이 아닌 올바른 general function family만을 찾도록 노력하면 됩니다.



## 6.2.1.1 Learning Conditional Distributions with Maximum Likelihood

5장에서 linear model에서 output을 gaussian으로 가정했을떄 MLE, MSE가 같다는 것을 보였습니다, 하지만 실제로 이것은 linear 모델이 아닌 any model $f(x;\theta)$ 에서도 성립합니다.

뉴럴네트워크를 디자인 할때 고려해야할 사항 중에 하나는 cost function의 gradient가 커야만 하며 learning 알고리즘을 가이드하기 위해 충분히 predictable? 해야 합니다. saturation(매우 평평한)이 있는 함수는 gradient가 작아지기 때문에 학습에 도움이 덜 되게 됩니다. 많은 경우 output or hiddent unit이 saturate될때 (output or hidden을 평평한 함수 즉 sigmoid같은 것을 사용했을때를 말하는 것 같다) <b>NLL 은 이러한 문제를 피하는데 도움을 줍니다. 대부분의 output unit은 exp 함수를 포함하고 있으며 이는 negative value 일 경우 매우 saturate된 값을 출력합니다. NLL에 있는 log 함수가 이러한 exp 때문에 포화되는 값들을 다시 되돌려 줍니다.</b>

MLE를 수행하기 위한 <b>cross-entropy cost를 사용에 대한 한가지 특징은 실제로사용되는 대부분의 모델들에 적용될때는 minimum value를 가지지 않는다는 것입니다.</b>(lower bound가 없어서.. 계속 loss를 떨굴수가 있다는 뜻인것 같다.) discreate ouput의 경우에 0 혹은 1의 확률을 나타내도록 설계되지는 않지만 임의적으로 가깝도록 할 수 있습니다.(???? 뭔소링니지 모르겠다.. 대충 받아들여보자면 discreate의 경우는 확률분포 혹은 확률밀도를 아웃풋으로 내는게 아니다 라는 느낌인데.) 로지스틱 회귀의 경우가 이러한 모델의 예입니다. real-value ouput의 경우 모델이 output 분포의 density를 조절할 수 있다면 train set output을 맞출 수 있는 형태로 매우 조밀한 밀도를 보이며 이 경우 cross-entropy는 negative infinity가 됩니다. 챕터7에서 설명될 regularization 방법들은 이러한 경우 모델이 무한대의 reward를 받지 않도록 하는 방법을 사용합니다.

## 6.2.1.2 Learning Conditional Statistics

$p(y|x;\theta)$ 를 학습하는 것 대신에 x가 주어졌을때의 conditional statistics를 학습 하는 것을 원할 수 있습니다. 예를 들어 y의 평균을 예측하는 $f(x;\theta)$를 원할 수 있습니다.

만약 충분히 복잡한 뉴럴네트워크를 사용한다고 가정하면 뉴럴네트워크를 일종의 wide class function의 representation으로 볼 수 있습니다. 그리고 이 function은 파라미터의 형태에 따라 결정되는 것보다는 continuity, boundedness라는 제약조건만을 가진 피쳐들로 제한되어 집니다. 이러한 관점에서 cost function을 단지 일종의 functional로 볼 수 있습니다. functional이란 function을 실수로 매핑하는 사상입니다. 이런 경우 learning은 파라미터를 선택하는 것이 아닌 적절한 function을 선택하는 것이 됩니다. cost functional을 우리가 원하는 특정 fucntion에서 minimum이 되도록 설계할 수 있습니다. 예를들어 cost functional이 $E[y|x]$ 에서 최소가 되도록 할 수 있습니다. 이러한 functional 에서의 optimization 문제를 풀기 위한 수학적 방법을 <b>calculus of variations</b> 이라고 하며 19장에 설명되어져 있습니다. 이 챕터를 이해하기 위해 모든것을 이해할 필요는 없으며 지금은 2가지 결과가 유도된다는 것만 이해하면 됩니다.

- #### $f* = argmin_{f}E_{x,y \sim  p_{data}} ||y-f(x)||^2$ 에 관한 optimization 문제를 caculis of variations으로 풀면 $f*(x) = E_{y\sim p_{data}(y|x)}[y]$ 가 유도됩니다.

즉 true data generating distribution에서 무한한 샘플을 가지고 학습할 수 있다면 MSE cost는 각 x에 대한 y의 평균을 예측하게 된다는 뜻입니다.

- ####  $f* = argmin_{f}E_{x,y \sim  p_{data}} ||y-f(x)||_1$ 이 수식을 calculus of variation으로 풀면 x에 대한 y의 median 값을 예측하는 함수를 얻을 수 있습니다. 이러한 cost function은 <b>mean absolute error</b> 라고 불립니다.

하지만 이러한 MSE, MAE 는 그라디언트를 기반으로하는 최적화 에서는 잘 작동하지 않습니다. 몇몇의 output unit은 위의 cost fucntion과 함께 쓰일대 매우 작은 그라디언트를 얻게 됩니다. <b>이것이 전체의 분포인 $p(y|x)$를 추정하는 것이 아닐때도 cross-entropy가 MSE, MAE보다 더 자주 쓰이는 이유중에 하나입니다.</b>



## 6.2.2 Output Units

## 6.2.2.1 Linear Units for Gaussian Output Distributions

가장 간단한 output unit은 non-linear가 없고 단순한 affine transformation 인 unit이며 이는 linear unit이라고 불립니다.

h라는 feature가 주어졌을때 linear output은 다음과 같이 계산됩니다. $\hat y = W^T h + b $ Linear output lear는 conditional gaussian distribution의 평균을 예측할때 종종 사용됩니다. 

## $p(y|x) = N(y; \hat y ,I)$

위와 같은 모델에 대한 maximize log-likelihood는 minimize MSE와 동치인걸 전에 살펴봤습니다. MLE 를 사용하여 Gaussian의 공분산을 학습하는 것도 가능하며 혹은 공분산을 입력으로 받아 평균을 예측하도록 할 수 있습니다.(given corvariance predict mean) 하지만 공분산은 모든 인풋에 대해 positive definite여야 하며 이러한 제약조건은 linear output layer로는 만족하기 어렵습니다. 그래서 공분산을 모델링 할때는 linear가 아닌 다른 output layer가 사용되곤 합니다. 

<b> linear unit은 saturation이 없기 때문에 그라디언트 기반 최적화 알고리즘에서 어려움을 덜 겪게 됩니다. $

## 6.2.2.2 Sigmoid Units for Bernoulli Output Distributions

많은 task들이 binary variable y를 예측하는 것을 필요로 합니다. 2개의 클래스에 대한 classfication이 이러한 형식을 따릅니다.

이 문제에 대한 maximum likelihood 접근방법은 $p(y|x)$를 베르누이 분포로 정의합니다. 베르누이 분포는 하나의 실수로 의해 정의됩니다. 뉴럴네트워크는 오직 p(y=1|x) 만을 예측하면 됩니다. 이 실수가 확률이 되기 위해서 0~1 안에있어야 한다는 제약조건이 붙습니다.

이러한 제약조건을 잘 설계하기위해선 몇가지 노력이 필요합니다. linear unit을 사용한다고 가정해봅시다. 0~1 의 값을 얻기 위해서 threshold를 통해 다음과 같은 식을 얻을 수 있습니다.

## $p(y=1|x) = max[0, min[1, w^Th + b]]$

<b>위의 수식은 적절한 분포가 되지만 이는 gradient desecent로 효과적으로 학습이 불가능 합니다. linear unit의 값이 0~1 범위를 넘어갈 때 마다 파라미터에 대한 gradient는 0이 되며 이는 해당하는 파라미터에 대한 업데이트를 불가능하게 해 문제가 됩니다. </b>

위의 분포를 모델링 하기 위한 적절한 방법은 잘못된 답을 할때마다 항상 강력한 gradient를 가지도록 하는 것입니다. 이는 maximum likelihood와 sigmoid output unit을 결합하는 것으로 효율적으로 모델링 가능합니다.

sigmoid output unit은 다음과 같이 정의됩니다.

## $\hat y = \sigma (w^Th + b) $

sigmoid output unit을 두부분으로 나눠서 생각 할 수 있습니다. 첫째는 $z=w^Th +b$ 의 linear layer 이며 두번째는 z를 확률로 만들어 주는 sigmoid activation 부분입니다.

잠시 z라는 변수를 사용하여 sigmoid function이 어떻게 확률 분포를 정의하는지 살펴봅시다. sigmoid 는 원래 unnormalized 확률분포 $\tilde{P}(y)$ 를 구성하기 위해 고안되어졌습니다. unnormalized라는 말은 합해서 1이 되지 않는다는 것을 뜻합니다. 이러한 unnormalized probability distribution에 적절한 상수를 나눠줌으로써 유효한 확률 분포를 만들 수 있습니다. 먼저 unnormalized log probability  $log \tilde{P}(y)$가 y,z에 대해 선형이라고 가정해 봅시다. log를 없애기 위해 exponential 함수를 적용 할 수 있습니다. 그 후 normalize 해주면 이는 sigmoidal transfromation of z에 의해 조절되는 일종의 베르누이 분포를 얻을 수 있습니다. (이 함수는 z,y를 받아 0~1 범위의 하나의 값을 내는 함수이므로 이를 베르누이의 파라미터 p로 생각하면 적절한 베르누이 분포를 생성하는 함수이라고 볼 수 있다)

exponentiation, normalization을 기반으로하는 확률분포는 statistical modeling에서 자주 사용되는 것입니다. 이러한 binary variable에 대한 z 변수를 보통 <b>logit</b> 이라고 부릅니다. 

이러한 log-space에서의 확률을 예측하는 접근은 maximum likelihood learning에서도 자주 사용됩니다. maximum likelihood를 사용하는 cost function이 $-logp(y|x) $ 이기 떄문에 cost function안의 log가 sigmoid의 exponential 때문에 사라지게 됩니다. 이러한 효과가 없다면 sigmoid의 saturation 되는 부분이 그라디언트 기반 학습을 방해할 것입니다. sigmoid로 파라미터라이즈된 베르누이 분포의 maximum likelihood loss function은 다음과 같습니다.

$J(\theta) \\
= -log P(y|x) \\
= -log \sigma ((2y-1)z) \\
= \zeta((1-2y)z)$

위와 같은 유도는 3.10섹션에서의 몇가지 props를 이용할 수 있게 합니다. loss function을 softplus function으로 다시 작성 함으로써, (1-2y)z 가 very negative일 때만 saturation이 발생한다는 것 알 수 있습니다. 즉 saturation은 옳은 답일 때 발생합니다. (y=1 이며 z가 very positive, y=0 이며 z가 very negative) , z가 틀린 답일 때(y=1 이며 z가 very negative, y=0이며 z가 very positive)  softplut function (1-2y)z= -z or z 이므로 |z| 로 간단히 나타낼 수 있습니다.  z가 잘못되었을때 |z| 는 매우 커지게 되며 softplus function은 |z| 을 작게 만들기 위한 방향으로 수렴하려고 합니다. z에 대한 미분은 sign(z)에 수렴하며, 즉 이 의미는 완전히 틀린 z를 가지고 있더라도 softplus function의 gradient가 수축하지 않는다는 것입니다. 이러한 특성은 gradient-based lerning에서 잘못된 z를 빠르게 고치도록 한다는 것을 의미합니다.

<img src='https://qph.fs.quoracdn.net/main-qimg-4d9834b37a3705da3332ff6af6f6eaaf' width='420' height='320' />

MSE같은 다른 loss function을 사용한다면 $\sigma (z)$가 saturation될 때마다 loss도 saturation 되게 됩니다. sigmoid activation은 위에서 봤듯이  옳은 답일 때 발생하며 (y=1 이며 z가 very positive, y=0 이며 z가 very negative) 이러한 옳은 일이 발생할때 gradient가 작아지게 됩니다. 이러한 이유 때문에 sigmoid output unit을 학습시키기 위해서 MLE가 대부분의 경우 선호됩니다.

해석적으로는 sigmoid function의 value는 open interval (0,1) 로 제한되기 때문에 항상 logarithm이 정의되며 finite 합니다. 반면 [0,1] closed interval로 정의하게 되면 logarithm이 infinite 한 경우 (0 인 경우) 가 생길 수 있습니다. 소프트웨어에서는 수치해석적 문제를 피하기 위해 negative log-likelihood 로 정의하는 경우가 많습니다. sigmoid function이 0으로 underflow되면.. logarithm of $\hat y$가 negative infinity 가 되게 됩니다. (이것은 무슨 의미이지??)



## Sigmoid output unit

- 베르누이 분포를 설계하는 자연스러운 방법. unnormalized log probability를 linear로 가정
- 잘못된 답일 때 항상 large gradient이며 . saturation은 아주 옳은 답을 할때만 발생.
- log likelihood를 사용하면 sigmoid의 exp를 벗겨줘서 saturation을 줄여준다.... 나머지 loss function은 학습이 잘 안될 수 있다.
- logarithm도 해석적으로 잘.. 정의된다.

## 6.2.2.3 Softmax Units for Multinoulli Output Distributions

binary variable의 경우에 우리는 하나의 값 $\hat y = p(y=1|x)$ 를 예측하도록 설계하였습니다. 이 숫자는 0~1 사이에 있기를 원하며 이는 log-likelihood를 위한 gradient based optimization이 잘 작동하기를 원하기 때문이였습니다. 우리는 대신에 z 라는 log of unnormalized probability를 예측하도록 하였고 이를 exponential, noramlize 함으로써 sigmoid function에 의해 조절되는 베르누이 분포를 얻을 수 있었습니다.

n개의 값에 대한 discreate variable의 경우로 일반화 하기 위해 $\hat y_i  = P(y=i |x) $ 인 <b>$\hat y$</b> 를 필요로 합니다. 이 경우 각 $\hat y_i$ 가 0~1 사이의 값이여야 하며 전체의 합이 1이여야 유요한 확률분포를 나타냅니다. 베르누이 분포에 했던것 처럼 multinoulli 분포에 대한 일반화를 할 수 있습니다. 첫번째로 linear layer가 unnormalized log probability를 예측하도록 합니다.

## $z = W^Th + b$

여기서 z_i 는 unnormalized log probability 이며. log를 없애기 위해 exponential 을 취하고 normalize를 해줍니다. 이를 통해 softmax function을 구할 수 있습니다.

## $ softmax(z)_i = \frac{exp(z_i)}{\sum_j exp(z_j)}$ 

logistic sigmoid 때뫄 마찬가지로 softmax의 exp 부분은 maximum log-likelihood를 사용하여 학습할때 잘 작동합니다. 이 경우 우리는 $log P(y=i;z) = log softmax(z)_i$ 를 최대화 하려고 합니다. 

## $log softmax(z)_i = z_i - log \sum_j exp(z_j)$

위의 수식은 z_i가 cost function에 대한 올바른 답을 항상 주는 것을 볼 수 있습니다. 위의 식은 saturation이 일어나지 않는데 이는 <b>두번째 텀이 매우 작아도 z_i 가 cost에 잘 기여하는 것을 볼 수 있습니다. </b> maximizing log likelihood를 수행할때 , 첫번째 텀은 z_i가 커지도록 하며 , 두번째 텀은 모든 z_i의 합이 작아지도록 하려고 합니다. 두번째 텀을 이해하기 위해 $log \sum_j exp(z_j)$ 이 거의 $max_j z_j$ 에 근사한다고 가정해 봅시다. 이러한 근사에 대한 생각은 exp(z_k) 가 어떤 z_k에 대해서도 $max_j z_j$에 대해서 더 작다는 것에 기반합니다. 이러한 근사를 통해 얻을 수 있는 해석은 NLL cost function이 항상 가장 incorrect prediction에 강한 패널티를 준다는 것입니다. 만약 옳은 답에 대해 lagest input of softmax를 가지고 있다면 $log \sum_j exp(z_j) \approx max_j z_j = zi$ 가 되어 z_i - z_i 가 되어 0에 근사할 것입니다. 이러한 경우(매우 옳은 경우) 전체의 cost에 별 영향을 주지 않게 됩니다. 

지금까지는 오직 한 example에 관해서만 이야기 했습니다. <b>전반적으로는 unregularzied maximum likelihood는 모델이 softmax가 train set안에서 관찰된 각각 outcome의 fraction of count를 예측하도록 합니다.</b>

## $softmax(z(x;\theta))_i \approx \frac{\sum _{j=1}^m1_{y^{(j)}=i,x^{(j)}=x}}{\sum _{j=1}^m1_{x^{(j)}=x}}$

maximum likelihood는 consistent estimator (train example이 무한하면 true에 converge) 이는 model family가 training distribution을 나타내기에 적절하다는 것을 보장합니다. 실무에서는 제한된 model capacity, 완벽하지 않은 optimization 때문에 위의 수식만을 근사하도록 모델이 학습됩니다.

log-likelihood가 아닌 다른 objective function은 softmax와 함께일때 잘 작동하지 않습니다. 특히 log가 없어 softmax의 exp를 없애지 못하는 objective 일 경우 학습이 잘 안됩니다. exp가 매우 negative일때 gradient는 vanishing되어집니다. 또한 MSE는 softmax unit에는 잘 작동하지 않으며 학습에 실패하게 됩니다. 이는 잘못된 예측에 매우 높은 확신을 가지는 경우라도 학습이 잘 안됩니다. (NLL의 경우엔.. 틀린답에 아주 강한 시그널을 줬는데.. MSE는 못한다) 왜 이런 경우가 발생하는지 이해하기 위해 softmax function 자체를 자세히 볼 필요가 있습니다.

sigmoid의 경우와 마찬가지로 softmax activation은 saturation이 발생할 수 있습니다. sigmoid function은 하나의 output을 가지며 input이 매우 negative or postivie일 경우에 saturation이 발생합니다. softmax의 경우에는 여러개의 output을 가지며 output value는 input value간의 차이가 매우 커질때 saturation이 발생합니다. softmax가 saturation이 발생할때 softmax를 기반으로하는 cost function 또한 saturation이 생깁니다.

softmax function이 다른 input 사이에서의 반응을 과낯ㄹ해 봅시다. softmax function은 모든 input z_i 에 상수 c를 더하는 것에 invariant합니다.

$softmax(z) = softmax(z+c)$

위의 특징을 사용하면 softmax의 numerically stable variant 를 유도할 수 있습니다.

$softmax(z) = softmax(z-max_i z_i)$

위의 수식은 z가 very postive or very negative일 경우에도 적은 수치적 에러를 가지고 softmax를 평가해볼수있게 합니다. <b>이 수식을 살펴보면 softmax가 max_i z_i에서 얼마나 벗어났는지에 따라 결정된다는 것을 알 수 있습니다.</b>

$softmax(z)_i$ 의 output은 input z_i가 maximal input일 때에 1로 saturation이 일어나며 이 의미는 z_i가 다른 z 들 보다 매우 크다는 의미입니다. 또한 output이 0으로 saturation될 때는 z_i가 maximum값이 아니며 max_i z_i가 매우 큰 값일때 발생합니다. 이는 sigmoid unit이 saturation 될때의 일반화된 버젼이며 sigmoid를 사용할때와 마찬가지의 learning difficulty 가 발생합니다. (NLL을 사용하지 않앗을때의... 문제점들)

softmax function의 z라는 인자는 두가지 방법에 의해 생성될 수 있습니다. 가장 많이 사용되는 방법은 이전의 레이어의 output이 각 z_i가 되도록 하는 것입니다. 즉 $ z = W^Th + b$ 로 나타낼 수 있습니다. <b>이러한 접근법은 분포를 지나치게 overparametrize 시킵니다. n개의 output의 합이 1이 되어야 한다는 것은 n-1 개의 파라미터만을 필요로 합니다. n-1 개의 파라미터를 알면 마지막 파라미터는 자동적으로 구해지게 됩니다. </b> 따라서 하나의 z_i가 고정되어야 한다는 조건을 부과 할 수 있습니다. 예를들어 z_n =0 이여야 한다고 합시다. 이는 정확하게 sigmoid unit이 하는 일입니다. $p(y=1|x) = \sigma(z)$ 는 $p(y=q|x)=softmax(z)_1$ 이며 2차원 z중에서 z_1=0인 경우와 동일합니다. n-1, n 개의 인자 모두 softmax가 같은 set of probability distribution을 가지도록 할 수 있습니다. 하지만 약간 다른 learning dynamic을 가집니다. 실무적으로는 overparameterized version, restricted version은 거의 차이가 없으며 overparameterized version이 구현이 간단해 더 자주 쓰입니다.

뇌과학의 관점으로는 softmax function이 일종의 unit 사이의 경쟁을 하도록 하는 방법이라고 생각 할 수 있습니다. softmax ouput의 합은 언제나 1이며 하나의 값이 커지면 다른 값은 작아져야 합니다. 이는 cortex에서 근처의 뉴런들끼리 억제하는 것과 매우 유사합니다. 과장하자면 이는 일종의 <b> winner-take-all</b> 의 형태가 됩니다. (maximal z_i와 다른 것들간의 차이가 클때 z_i의 output은 거의 1이고 나머지들은 거의 0 이다)

softmax라는 이름은 약간 헷갈립니다. 이는 max function 보다는 argmax function과 더 밀접한 관련이 있습니다. 'soft' 라는 이름은 softmax가 continuous, differentiable 한 것에서 나왔으며 , argmax 함수는 one-hot vector로 표현되며 이는 연속이 아니며, 미분불가능합니다. 그래서 softmax는 argmax의 soft version을 제공합니다. max function의 soft version은 $softmax(z)^Tz$ 에 대응하합니다. 이는 softmax가 softargmax로 불리는게 나을지 모릅니다. 현재의 이름은 일종의 관습처럼 되었습니다.



## Softmax output unit

- multinoulli 분포를 모델링하는 적절한 방법. sigmoid와 마찬가지로 z_i를 unnormalized log probability로 정의. 
- $log softmax(z)_i = z_i - log \sum_j exp(z_j)$ 를 생각해보면 하나의 z_i가 매우커서 $log \sum_j exp(z_j)$ 이 z_i 에 근사한다고 생각해보자. 만약 z_i가 매우 옳은 답이면 z_i - z_i =0 이 되어서 loss에 별 영향을 안주게 되고 incorrect prection에는 강한 패널티를 준다
- softmax는 일종의 fraction of outcome 을 학습한다고 한다.(잘 이해가 안감)
- sigmoid와 마찬가지로 NLL(혹은 Loss에 exp가 없는 경우)가 아닌경우에 학습에 문제가 발생할 수 있다.
- input z_i 간의 차이가 매우 커질때 saturation이 발생한다. 특히 softmax(z_i) 에서 z_i가 maximal input일때 1로 saturation, z_i가 maximum이 아니며 maximum값이 매우 클때 0으로 saturation 이 발생합ㄴ디ㅏ. 
- 사실 파라미터가 n-1개만 있으면 되는데.. n개로 모델링 해도 되는데 약간 다른 learning dynamic 즉.. 하나의 값은 고정되버릴려고 하는 dynamic?을 가진다 근데 n개로 모델링하는게 더 간단해서 자주쓰인다.
- 뇌과학적 관점으로는 인접한 뉴런들이 서로 억제하는 것과 비슷하다.
- softmax라는 이름보다는 softargmax라고 보는 것이 더 이해하기 쉽다.


## 6.2.2.4 Other Output Types

일반적으로 $p(y|x;\theta)$를 정의하면 maximum likelihood는 $-log p(y|x;\theta)$를 cost fucntion으로 자동적으로 제안해 줍니다.

일반적으로 뉴럴네트워크를 $f(x;\theta)$에 대한 representation으로 생각 할 수 있습니다. $f(x;\theta)$에 대한 출력 자체가 y에대한 예측이 아니며 대신에 $f(x;\theta)=\omega$ 를 통해 y의 분포에 대한 파라미터를 예측하도록합니다. 그래서 우리의 loss fucntion은 $-log p(y|;\omega(x))$ 가 됩니다. 

예를들어 $p(y|x;\theta)$ 이라는 conditional gaussian의 분산을 학습하고 싶다고 합시다. 가장 간단한 경우는 분산이 상수인 경우입니다. 이 경우 MLE를 통해 closed form으로 풀수 있으며  $E[||y - \hat y||^2]$ 를 통해 estimator의 분산을 간단히 구할 수 있습니다. 연산량이 많이 들지만 특별한 경우에대한 코드를 작성할 필요가 없는 방법은 $p(y|x)$ 라는 분포가 분산을 표현하도록 하는 것입니다. 즉 $p(y|x)$ 라는 분포는 $f(x;\theta)=\omega$에 의해 조정되게 됩니다. NNL인 $-log p(y|;\omega(x))$ 은 cost function을 제공해 주게 되고 otpimization을 통해 분산을 서서히 학습하게 됩니다. <b>입력과 y라는 분포의 표준편차가 상관이 없는 가장 간단한 경우에는 $\omega$를 단순히 복사를(바로 사용함으로써?) 함으로써 새로운 파라미터를 만들 수 있습니다. (?뭔소리야..? y의 분산이 인풋과 관련이 없다면 네트워크의 아웃풋을 분산으로 바로 쓸수 있다는 소리같다)</b>

이 새로 만들어진 파라미터는 $\sigma$ 그 자체가 되거나 혹은 $\sigma^2$를 나타내는 v 혹은 $\frac{1}{\sigma^2}$를 나타내는 $\beta$가 될 수 있고 이는 분포를 어떻게 파라미터화 하였는지에 따라 달라집니다. 각기 다른 x에 대해 y의 분산의 양을 예측하기를 원한다면 이는 <b>heteroscedastic(이분산성)</b> 이 됩니다. 이러한 이분산성의 경우에 간단하게  $f(x;\theta)$의 아웃풋중 하나의 값을 분산으로 사용하면 됩니다. 가우시안 분포에 대해 이러한 것을 하는 전형적인 방법은 variance가 아닌 precision을 통해 분포를 추정하는 것입니다. 

3.22에 따르면 다른 파라미터에 따른 pdf를 자주 평가해야 한다면 beta를 통해 분포를 파라미터화 하는 것이 더 효율적이다. (?왜지? 그냥.. inverse를 하지 않아도 되기 때문인 것 같다.. 공분산행렬은... pdf를 구할때 인버스를 해줘야 한다. 만약에 .. 풀랭크가 아닌 경우도 있을 수 있기도 하고.. 연산할때 역행렬 안해도 되기 떄문인듯) multivariate의 경우에 diagonal precision matrix가 보통의 경우에 사용됩니다. $diag(\beta)$

## Modeling Gaussian covariance network

### 이 밑의 부분들은 모두 Gaussian의 covariance or precision 매트릭스를 모델링 할때의 이야기들이다.

위와 같은 모델링은 gradient 기반의 방법에서 잘 작동합니다. $\beta$에 의해 파라미터화된 가우시안 분포의 log-likelihood는  오직 $\beta_i$에 대한 곱과 $log \beta_i$만을 더하는 것만을 포함합니다. 곱, 더하기, log에 대한 gradient는 잘 동작합니다. 만약에 우리가 output을 $\sigma^2$로 파라미터화 하였다면 나눗셈을 해야만 합니다. 나눗셈은 0근처에서 이상하게 매우 가파르게 됩니다. 
## 1/x
<img src='http://cfile6.uf.tistory.com/image/2212FD3855D7FBCE055308' width=320, height=400>

큰 gradient가 learning에 도움을 주지만 매우 큰 gradient는 안정적이지 않게 합니다. output을 표준편차를 통해 모델링 할때는 log-likelihood가 여전히 나눗셈을 포함하고 제곱도 포함하게 됩니다. 제곱이라는 연산의 gradient는 0근처에서 vanishing되며 이는 파라미터를 학습하는 것을 어렵게 합니다.

## square of x
<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/7/76/Parabola2.svg/240px-Parabola2.svg.png' width=240, height=360>

## 가우시안 분산을 모델링 할때 $\sigma ^2$, $\sigma$ ,$\beta$ 

- $\sigma ^2$ 을 예측하는 모델을 사용한다면 MLE에 나눗셈이 들어가는데.. 이는 0근처에서 매우큰 gradient 
- $\sigma$ 를 예측하도록 모델링하면 MLE에 나눗셈이 들어가고, 제곱이 들어가는데 나눗셈도 안좋고 제곱은 0 근처에서 vanishing gradient
- $\beta$를 통해 모델링 하면 +, X, log만 들어가게되어 gradient based 알고리즘에서 효율적이다


표준편차, 분산, precision 중 어떤것을 사용하는지와 상관 없이 가우시안의 공분산 행렬이 positive definite(대칭행렬이고,, 고유값이 모두 0초과)해야 합니다. precision matrix의 고유값들은 서로 관련이 있으므로(precision matrix가 공분산행렬의 역행렬 ) 이므로 precision matrix 또한 positive definite 해야만 합니다. 

### precision, covariance matrix는 positive definte해야한다.

대각행렬 혹은 대각행렬의 스칼라배를 사용한다면 output이 만족해야하는 유일한 조건은 positive 여야 한다는 것입니다. 만약 a를 diagonal precision matrix를 결정하기 위한 네트워크의 아웃풋으로 사용한다면 우리는 positive precision matrix를 얻기 위해 softplus function을 사용 할 수 있습니다. 이러한 방법은 분산, 표준편차를 사용할때도 동일하게 적용됩니다. 

<img src='https://qph.fs.quoracdn.net/main-qimg-4d9834b37a3705da3332ff6af6f6eaaf' width=240, height=360>

### precision matrix를 대각행렬로 가정하면 positive output으로만 매트릭스를 채워넣으면 된다. 이때 softplus를 쓰면 적절하다




covariance, precision matrix를 대각행렬이 아닌 것을 사용하는 것은 매우 드뭅니다. 만약 공분산행렬을 full rank, conditional 로 가정하면 파라미터화 하는 방법이 공분산 행렬이 positive-definite 한 것을 보장해야 합니다. 이는 $\sum(x) = B(x)B^T(x)$ 로 모델링 하며 B는 제약이 없는 정방행렬이 됩니다. 

여기서 하나의 문제는 공분산 행렬이 full rank라면 likelihood를 계산하는 연산량이 매우 비싸지게 되고 dxd 공분산행렬의 행렬식, 역행렬을 계산하는 것이 O(d^3)이 되게 됩니다.

### full rank 공분산 행렬을 모델링 하는 것은 정방행렬을 통해 가능하지만.. likelihood를 계산할때 행렬식, 역행렬에 대한 연산량이 많아지게 된다.. 

## modeling with mixture density network

우리는 종종 multimodal regression을 수행하곤 합니다. 이는 조건부 확률 분포 $p(y|x)$에서 부터 어떤 값을 예측하려고 하며 y의 공간이 여러개의 봉우리들이 존재 할 수 있습니다. 이 경우 output 을 Gaussian mixture를 통해 모델링 하는 것이 적절합니다.

뉴럴네트워크를 통한 Gaussian mixture 방법을 보통 <b>mixture density network</b> 라고 합니다. n개의 Gaussian이 mixture된 분포는 다음과 같이 정의됩니다.

## $p(y|x) = \sum_{i=1}^n p(c=i|x)N(y;\mu^{(i)}(x), \Sigma^{(i)}(x))$

네트워크는 세가지 아웃풋을 가져야만 합니다. $p(c=i|x)$ 를 정의하는 vector, 와 i번째 mean, variance를 예측해야만 합니다. 이들 output은 각각 다른 제약조건을 가집니다.

- mixture components $p(c=i|x)$ : 이것의 형태는 일종의 latent variable c에 관련된 n개의 요소를 가진 multinoulit distribution입니다. 이는 전형적으로 n차원 vector에 대한 softmax로 얻어질 수 있습니다. 즉 output이 positive 이며 총 합이 1이여야 한다는 제약이 있습니다.

- Mean $\mu ^{(i)}(x)$ : 이는 i번째 가우시안 분포에 대한 중심 혹은 평균을 나타냅니다. 이에는 제약조건이 없습니다. (제약이 없지만 전형적으로는 mean을 위한 output unit으로는 nonlinearlity를 사용하지 않습니다.) y가 d차원 vector라면 네트워크의 output은 nxd 매트릭스이며 d-차원 y에 대한 모든 n개의 가우시안 분포의 평균을 출력합니다. mixture gaussian의 mean을 학습하는것은 하나를 학습할때 보다 좀 더 어렵습니다. 우리는 observation으로 학습되는 평균이 실제 분포이길 바랍니다.(true distribtution이길 바란ㄷ) 실제로는 각 observation이 어떤 component에 속하는지 알 수 없습니다. NLL의 표현은 근본적으로 각 example이 각 component의 loss에 기여하는 정도를 가중한 형태입니다. 이러한 가중합은 각 component에 example이 속할 확률을 통해 가중치가 매겨지게됩니다.

- Covariance $\Sigma^{(i)}(x)$ : 이는 각 component i에 대한 공분산 매트릭스를 정의합니다. 하나의 가우시안 분포를 학습할때 보통 행렬식 계산을 피하기 위해 대각행렬을 사용합니다. mixture의 mean을 함께 학습해야 하기 때문에 MLE는 각 point에 대한 부분적인 책임을 각 mixture component에 대해 할당해야 하기 때문에 더 복잡해집니다. gradient descent는 mixture model에 대한 올바른 NLL 이라면 적절하게 동작합니다.

mixture density network를 gradient descent로 학습할때 잘 되지 않는다는 것이 보고되었습니다. 이런 경우 한가지 이유는 분산에 의한 division이며 하나는 수치적 문제입니다. (특히 어떤 example에 의해 분산이 매우 작아지면 매우 큰 gradient를 산출합니다.) 

한가지 해결방법은 clip gradient 이며 하나는 휴리스틱하게 gradient를 scale 하는 것입니다. 

## 6.3 Hidden Units

여기서 설명하는 모든 hidden unit들이 모든 point에서 미분가능 하지 않습니다. 예를 들어 g(z) = max(0,z) 라는 relu 는 z=0에서 미분 불가능합니다. 이는 gradient descent를 사용하지 못하는 것으로 생각 할 수 있습니다. 하지만 실무적으로는 gradient descent가 여전히 작동하며 특정한 task를 충분히 잘 수행합니다. <b> 이는 뉴럴네트워크의 학습 알고리즘이 cost function의 local minimum에 도달하는 것을 보장하지는 않지만 cost function을 매우많이 줄이는 방향으로 학습된다는 점 때문에 가능합니다.</b> 우리가 실제로 gradient가 0이 되는 지점으로 학습 될것이라고 기대하지 않기 때문에 cost function의 minima가 undefined gradient point에 할당되는 것도 수용할 만 합니다. 

미분불가능한 hidden unit들은 보통 small number of point 에서만 미분 불가능한 경우가 많습니다. 어떤 함수가 미분가능하다는 것은 좌미분가능, 우미분 가능하며 그 값이 동일하면 미분가능하다고 합니다. 뉴럴네트워크에서 사용되는 함수들은 보통 좌미분, 우미분은 정의가 되어져 있습니다. relu의 경우네는 좌미분은 0, 우미분은 1이 됩니다. 뉴럴네트워크 라이브러리에서 보통 undefined gradient point에서 에러를 내기 보다는 좌미분, 우미분 중 하나를 선택하여 사용합니다. 이는 디지털 컴퓨터에서의 gradient-based optimization에서는 항상 수치적 에러가 존재한다는 것으로 정당화 될 수 있습니다. 즉 뉴럴네트워크에서 한 unit이 g(0)을 계산하려고 할때 이는 수치적 bite의 제한으로 0이 아닌 true 0일 경우는 매우 적습니다. 몇몇 문헌에서는 더 이론적인 정당화가 가능하지만 이는 뉴럴네트워크 학습에는 잘 사용되지 않습니다. 

### Relu는 0에서 미분이 불가능하지만.. 실무적으로 그냥 쓴다.  


## 6.3.1 Rectified Linear Units and Their Generalizations

Relu는 linear units과 매우 비슷하기 때문에 최적화 하기 쉽습니다. linear unit과 relu와의 다른 점은 relu의 정의역의 반의 output은 0이 되는 것입니다. 이는 relu의 도함수가 unit이 active될때마다 크게 유지되도록 합니다. gradient가 클 뿐만 아니라 매우 consistent하게 됩니다. relu의 이차도함수는 0 almost every where이며 active relu의 일차 도함수는 1 everywhere 입니다. 이 의미는 2차 형식을 도입한 activation fucntion들 보다 gradient의 방향이 학습에 더 유용하다는 의미입니다.

- <b>4.3.1 Beyond the Gradient: Jacobian and Hessian Matrices</b> : 이차 도함수는 input의 변화에 따라 일차 도함수가 얼마나 바뀌는지를 말해줍니다. 이는 gradient 만을 기반으로 optimization을 할때 어느정도의 improvement를 기대할 수 있을지를 말해줍니다. 이차 미분을 일종의 curvature의 measure로 생각 할 수 있습니다. 어떤 이차형식의 꼴 (많은 함수들이 이차형식이 아니지만 실제로는 이차형식으로 근사하여 나타낼 수 있습니다.) 이 함수가 이차 도함수가 0이라면 curvature가 없다는 뜻입니다. 이는 완벽히 flat line이며 이의 값이 gradient만으로도 예측 가능하다는 뜻입니다. 만약 gradient가 1이라면 epsilon 만큼 negative gradient step을 밣는다면 cost function이 epsilon 만큼 줄어들게 된다는 것을 의미합니다. 만약 이차미분이 음수라면, curvature이 오목한 모양이라는 뜻이며 이는 cost function이 epsilon보다 더 많이 감소할 수 있다는 뜻입니다. 마지막으로 이차미분이 positive 라면 볼록한 형태이며 epsilon 보다 적게 cost가 감소 할 수 있다는 의미입니다.

### 즉 1, relu는 이차미분이 0 almost every where 이며 gradient만으로 이론적으로 완벽히 최적화가 가능하며 gradient step만큼의 cost decrease를 기대할 수 있다. 2, active일때의 일차 미분이 1 이여서 항상 일정 수준의 gradient step을 기대할 수 있다.



relu는 보통 affine transform의 뒤에 붙여서 사용합니다.

## $h = g(W^Tx + b) $

파라미터를 초기화 할때 보통 많이 사용하는 것은 b를 0.1과 같은 small, positive value로 설정합니다. 이는 처음에는 relu의 대부분의 입력들이 active 하도록 하여 미분이 가능하도록 하기 위해서입니다.

<b>relu의 단점은 activation이 0인 example에서는 gradient based 방법으로 학습이 불가능 하다는 것입니다</b> relu의 다양한 변종들이 every where gradient를 계산가능 하도록 일반화 합니다.



## 6.3.2 Logistic Sigmoid and Hyperbolic Tangent

<img src='https://cdn-images-1.medium.com/max/1600/1*f9erByySVjTjohfFdNkJYQ.jpeg' width=240, height=360>


- sigmoid는 이전에 살펴봤듯이 very positive, negative 일때 satuaration을 보인다. 이것이 gradient based learning에서 어려움을 겪게하며 특정한 이유가 있지 않는이상 잘 안쓴다

- tanh는 sigmoid와 거의 비슷한데. sigmoid보다 더 낫다고 알려져 있다. tanh는 identity function과 sigmoid 보다 더 닮아 있다.예를 들면 tanh(0)=0 이지만 sigmoid(0) =1/2 이다. 이러한 이유 때문에 딥뉴럴넷을 학습할때 더 잘된다.  $\hat y = w^T tanh(U^T tanh(V^Tx)) $을 학습하는 것이 $w^TU^TV^Tx $를 학습하는 것과 비슷하다(sigmoid 보다는..?) 이 때문에 tanh가 학습을 쉽게 하도록 한다.

## 6.3.3 Other Hidden Units

- Linear Activation : 파라미터를 줄이는데 유용한 경우가 있다. $h=g(W^Tx +b)$ 하나의 레이어로 표현할떄의 파라미터는 np (n=input, p=output)인데 $h=g(V^TU^Tx +b)$ 두개의 레이어로 표현하면 (n+p)q (q가 U의 아웃풋일떄) 가 된다. 작은 q를 사용하게 된다면 파라미터를 줄이는 효과가 있다. 이는 linear transformation에 low-rank의 제약을 주는 것이다.

- Softmax Activation : 종종 switch 혹은 attention의 개념이 들어갈때 사용한다.

- Radial basis function : $W_{i,i}$ 라는 template이 x에 더 가까워지도록 한다. x에 가장 가까워 졌을대 saturation이 일어난다. 하지만 최적화가 힘들다

- Softplus : relu의 smooth version 으로 볼수 있습니다. Glorot은 relu보다 softplus가 더 좋은점을 분석하였는데 실제로는 매우 성능이 떨어집니다. 이는 hidden unit의 성능이 매우 비직관적이라는 것을 보여줍니다. softplus는 relu에 비해 everywhere 미분가능하며 saturation도 적게 일어나지만.. 실제로는.. 잘 작동하지 않습니다.

<img src='https://qph.fs.quoracdn.net/main-qimg-4d9834b37a3705da3332ff6af6f6eaaf' width='420' height='320' />

## 6.4.1 Universal Approximation Properties and Depth

### 더 깊은 모델을 선호하는 통계적 이유 

- 더 깊은 모델을 선택하는 것은 우리가 학습하고 싶은 함수가 몇몇 간단한 함수들의 composition이라는 것에 대한 prior(믿음)을 포함하고 있다


## 6.5 Back-Propagation and Other Differentiation Algorithms

back-propagtation 은 종종 뉴럴네트워크를 학습하는 learning 알고리즘으로 잘못 설명됩니다. 실제로는 back-propagtation은 오직 gradient를 계산하는 방법일 뿐이며 이러한 gradient를 사용하여 SGD와 같은 알고리즘이 learning을 수행합니다.

또한 back-prop에 대한 다른 오해는 이는 뉴럴네트워크에만 작동하느 것이 아닌 임의의 함수에 대한 도함수를 계산할 수 있습니다. (어떤 함수에대한 적절한 back-prop에 대한 response는 도함수가 정의되지 않았다 일수 있습니다) 예를들어 임의의 함수 f에 대한 gradient $\bigtriangledown _x f(x,y)$ 를 계산하고 싶다고 해봅시다.이때 x는 x에 대한 도함수를 구하고 싶고 y는 도함수를 구할 필요가 없는 input이라고 생각합시다. learning 알고리즘에서 우리가 필요한 gradient는 파라미터에 대한 cost function의 gradient를 필요로 합니다. 많은 머신러닝 테스크에서는 다른 종류의 도함수를 구하기도 하며 이는 learning process의 일종이거나 학습된 모델을 분석하기 위해서입니다. back-prop 알고리즘은 이러한 것에 모두 적용가능하며 파라미터에 관한 cost function의 gradient를 구하는것에만 제한되지 않습니다. 이러한 아이디어는 매우 일반적이며 여러개의 output을 가지는 f에 대한 자코비안을 계산하는 것같이 계산가능 합니다. 우리는 여기서 f가 하나의 output을 가질때로 제한하여 설명을 합니다.



## 6.5.1 Computational Graphs

지금까지는 뉴럴네트워크를 설명하기 위해 informal graph language를 사용했습니다. backprop을 정확히 설명하기 위해서는 보다 정확한 computational graph language를 사용하는 것이 도움이 됩니다.

computation을 그래프로 표현하는 많은 방법들이 존재합니다. 우리는 graph의 각각 node 들이 일종의 variable을 나타내며 이 variable은 scalar, vector, matrix, tensor 혹은 다른 종류의 변수들일 수 있다고 생각합니다.

graph를 나타내기 위해서 operation이라는 아이디어가 필요합니다. operation은 하나 혹은 더 많은 variable들에 대한 간단한 함수입니다. graph langugae는 일종의 허용된 operation들의 집합입니다. operation보다 더 복잡한 function은 일종의 다양한 operation들의 composition으로 설명될 수 있습니다.

일반성을 잃지 않고 operation을 하나의 output variable을 출력하는 것으로 정의합니다. 이렇게 할 수 있는 이유는 하나의 output variable이 벡터와 같이 여러개의 요소들을 가지는 varible이 될 수 있습니다. back-prop의 소프트웨어 구현에서는 보통 multiple output의 operation들을 지원하지만 여기서는 설명을 위해 하나로만 가정합니다.

variable y가 variable x에대해 operation을 적용하여 계산된 것이라면 x에서 y로의 edge를 그립니다. 때떄로는 output node에 이름을 달기도 합니다. operation이 context로 부터 명확한 경우 이러한 이름을 달지 않는 경우도 있습니다.

- variable : 임의의 type을 가지는 변수
- operation : variable에 대한 간단한 함수
- function : operation들의 모음

## 6.5.2 Chain Rule of Calculus

chain rulue은 도함수가 알려진 함수들의 조합으로 이뤄진 함수의 도함수를 계산할때 사용하는 바업입니다. back-prop은 chain rule을 계산하는 알고리즘로써 매우 효율적인 연산량을 가집니다. 

x를 실수로 가정하고, f,g를 실수를 실수로 사상하는 함수라고 가정한다면 chian rulue은 다음과 같습니다.

## $\frac{dz}{dx} = \frac{dzdy}{dydz}$

이를 scalar 의 경우를 넘어선 경우로 일반화를 할 수 있습니다. 

$x \in \mathbb{R} ^m , y \in \mathbb{R} ^n $로 가정하고 g를 m차원에서 n차원으로의 매핑, f를 n차원에서의 1차원으로 매핑이라고 생각합시다. 또한 y=g(x), z= f(y) 로 가정하면 

## $\frac{\partial z}{\partial x_i}= \sum_j \frac{\partial z}{\partial y_j}  \frac{\partial y_j}{\partial x_i}$

왜 이렇게 나오는 지를 생각해보자...<img src='http://1.bp.blogspot.com/-i_J2TtU1u40/UqgccCnyf1I/AAAAAAAACFw/LMTBTLfqgf0/s320/20131211_170319.png' >

모든건.. 자코비안 매트릭스에서,,, 생각될 수 있음


아웃풋이 스칼라일 경우 자코비안은 밑처럼 계산된다.

$
\frac{\mathrm{d} z}{\mathrm{d} \mathbf{x}} = \begin{bmatrix}
\frac{\partial z}{\partial x_1} &  \cdots & \frac{\partial z}{\partial x_m} 
\end{bmatrix}
$

하나의 x_i 즉 스칼라에 대한 미분은 다음과 같다. (Y가 벡터인것에 주의하자 이를 다시 자코비안매트릭스로 표현해야한다.)

$
\frac{\mathrm{d} z}{\mathrm{d} x_i} = \frac{\partial z}{\partial \mathbf{y}} \frac{\partial \mathbf{y}}{\partial x_i}
$
= 
$
\begin{bmatrix}
\frac{\partial z}{\partial y_1} & \cdots & \frac{\partial z}{\partial y_n} 
\end{bmatrix}
$
$
\begin{bmatrix}
\frac{\partial y_1}{\partial x_1}\\ 
\vdots\\ \frac{\partial y_n}{\partial x_1}
\end{bmatrix}
$
$
= \sum_j \frac{\partial z}{\partial y_j}  \frac{\partial y_j}{\partial x_i}$

$\bigtriangledown _{\mathbf{x}^z} = \frac{\mathrm{d} z}{\mathrm{d} \mathbf{x}} = \begin{bmatrix}
\frac{\partial z}{\partial x_1} &  \cdots & \frac{\partial z}{\partial x_m} 
\end{bmatrix} =  \begin{bmatrix}
 \begin{bmatrix}
\frac{\partial z}{\partial y_1} & \cdots & \frac{\partial z}{\partial y_n} 
\end{bmatrix}
\begin{bmatrix}
\frac{\partial y_1}{\partial x_1}\\ 
\vdots\\ \frac{\partial y_n}{\partial x_1}
\end{bmatrix}& , \cdots,& \begin{bmatrix}
\frac{\partial z}{\partial y_1} & \cdots & \frac{\partial z}{\partial y_n} 
\end{bmatrix}
\begin{bmatrix}
\frac{\partial y_1}{\partial x_1}\\ 
\vdots\\ \frac{\partial y_n}{\partial x_m}
\end{bmatrix} 
\end{bmatrix}$

내적은 스칼라이므로 스칼라의 트랜스포즈는 같다 각각 엘리먼트를 트랜스포즈해주자.

$=\begin{bmatrix}
\begin{bmatrix}
\frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_n}{\partial x_1} 
\end{bmatrix}
\begin{bmatrix}
\frac{\partial z}{\partial y_1}\\ 
\vdots\\ \frac{\partial z}{\partial y_n}
\end{bmatrix} & ,\cdots,  &
\begin{bmatrix}
\frac{\partial y_1}{\partial x_m} & \cdots & \frac{\partial y_n}{\partial x_m} 
\end{bmatrix}
\begin{bmatrix}
\frac{\partial z}{\partial y_1}\\ 
\vdots\\ \frac{\partial z}{\partial y_n}
\end{bmatrix} 
\end{bmatrix}$

$=\begin{bmatrix}
\frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_n}{\partial x_1}\\ 
 & \vdots & \\ 
\frac{\partial y_1}{\partial x_m} & \cdots & \frac{\partial y_n}{\partial x_m}
\end{bmatrix}\begin{bmatrix}
\frac{\partial z}{\partial y_1}\\ 
\vdots\\ \frac{\partial z}{\partial y_n} 
\end{bmatrix} = (\frac{\partial \mathbf{y}}{\partial \mathbf{x}})^T \bigtriangledown _{\mathbf{y}^z} 
$

위의 식으로부터 x라는 variable에 대한 gradient는 자코비안 매트릭스와 y에대한 z의 gradient의 곱으로 얻어질 수 있습니다. back-prop 알고리즘은 그래프 안의 각 operation들의 jacobian-gradient product 들로 구성되어져 있습니다. 

이러한 back-prop 알고리즘을 벡터 뿐만아니라 임의의 차원을 가진 tensor에 적용하는 것이 일반적입니다. 개념적으로 이는 vector일때의 back-prop 알고리즘과 동일합니다. 유일한 차이점은  각 element들이 어떻게 배열되어 텐서를 만들었는지만 다릅니다. 우리는 tensor에 대한 back-prop 전에 텐서를 flatten 하여 일종의 벡터로 만들수 있고 이에 관한 gradient를 계산할 수 있습니다. 다시 이를 tensor의 형태로 reshape 할 수 있습니다. 이러한 rearrange의 관점에서는 tensor의 경우에도 back-prop은 gradient에 jacobian을 곱하는 것 과 동일합니다.(결국 벡터의 경우와 동일하다)

tensor $\mathbf{X}$ 에대한 z의 gradient를 나타내기 위해서 즉 $\bigtriangledown _{\mathbf{X}^z} $ 를 나타내기위해서 , $\mathbf{X}$를 vector일때 처럼 나타냅니다. $\mathbf{X}$의 indices는 이제 multiple coordinate를 가질 수 있습니다. 예를들어 3D tensor의 경우에는 $\mathbf{X}_i = (x,y,z)$ 를 나타내게 됩니다. 이러한 i 번째 요소가 어떠한 tuple 을 나타내게 함으로써 추상화 할 수 있습니다. 모든 인덱스 i에 대해서  $(\bigtriangledown _{\mathbf{X}^z})_i $는 $\frac{\partial z}{\partial \mathbf{X}_i}$ 가 되게 됩니다. 이것은 벡터 일때 indices i 에 대한 gradient 가 $(\bigtriangledown _{\mathbf{x}^z})_i = \frac{\partial z}{\partial x_i}$ 인것과 비슷합니다.이런 노테이션을 사용함으로써 tensor에 대한 chain rule을 다음과 같이 쓸 수 있습니다.

## $ \bigtriangledown _{\mathbf{X}^z}= \sum_j (\bigtriangledown _{\mathbf{X}^{Y_j}}\frac{\partial z}{\partial Y_j})$

위의식은.. 도저히 내능력으로는 유도못하겠다..

## 6.5.3 Recursively Applying the Chain Rule to Obtain Backprop

chain rule 을 사용하면 어떤 scalar에 대한 gradient 즉 scalar를 출력하기 위해 사용된 모든 computational graph에 관한 어떠한 node에 대한 gradient를 구할 수 있습니다. 그러나 이러한 표현에 대한 실제적인 계산을 위해서는 몇가지 고려사항이 필요합니다.

특히, gradient를 위한 전체의 표현식중에 몇가지 subexperssion이 반복적으로 나타날 수 있습니다. gradient를 구하기 위한 과정은 이러한 subexpression을 저장할 것인지 혹은 다시 여러번 계산할 것인지를 결정해야 합니다. 이러한 예는 그림 6.9에 나와있습니다. 몇몇 경우에는 어떤 subexpression을 두번 계산하는 것은 연산의 낭비일 뿐입니다. 복잡한 그래프의 경우 이러한 subexpression을 두번 계산하는 일이 exponential 하게 발생할 수 있고. 즉 이는 naive한 구현은 chain rule을 실행 불가능하게 만들 수 있습니다. 어떤 경우에는 같은 subexperssion을 두번 계산하는 것이 실행중의 메모리 소모를 줄이기 위한 옳은 방법일 수 있습니다.

back-prop은 메모리에 상관없이 공통이 subexpression의 수를 줄이기 위해 설계되었습니다. node $u^{(j)}$ 부터 $u^{(i)}$($u^{(j)}$를 부모로 하는 unit) 의 edge를 단한번씩만을 방문하기 때문에 back-prop은 중복되는 subexpression의 지수적 증가에 대한 연산량을 회피 할 수 있습니다.

$\frac{\partial z}{\partial w} \\
= \frac{\partial z}{\partial y}\frac{\partial y}{\partial x}\frac{\partial x}{\partial w} \\
= f'(y)f'(x)f'(w) \,\,\,\, (6.52)\\
= f'(f(f(w)))f'(f(w))f'(w) \,\,\,\, (6.53)$

#### 결국 back-prop은 6.52 처럼 도함수를 계산하기 때문에.. sub-expression이 매우 많아져도 그래프의 edge에 linear 한 연산량을 얻을 수 있다.!! 하지만 y,x,w를 다 저장하고 있어야 하기 때문에 메모리는 많이 소모한다.!!

이 식이 어떻게 나왔을까..

## $\frac{\partial u^{(n)}}{\partial u^{(j)}} = \sum_{i:j \in Pa(u^{(i)})} \frac{\partial u^{(n)}}{\partial u^{(i)}} \frac{\partial u^{(i)}}{\partial u^{(j)}}$

$u^{(j)}$ 의 자식노드가 바로 $u^{(n)}$ 이라고 생각해보자 그렇다면...

$u^{(n)} $  
$= f^{(j)}(A^j) = f^{(j)}(\{ u^{(j)} | j \in  Pa(u^{(i)}) \})$

즉 A^j는 스칼라를 벡터로 매핑하는 사상이며 f^j는 벡터를 스칼라로 매핑하는 사상이다. 이때 A^j 즉... j번째 unit을 부모로 하는 애들이 m개가 있다고 해보자. 즉 스칼라를 벡터로 벡터를 다시 스칼라로 보내는 사상에 대한 미분을 생각해보면 summation이 나온다.

### $\frac{\partial u^{(n)}}{\partial u^{(j)}} = \frac{\partial u^{(n)}}{\partial \mathbf{A^{(j)}}} \frac{\partial \mathbf{A^{(j)}}}{\partial u^{(j)}} = \begin{bmatrix}
\frac{\partial u^{(n)}}{\partial \mathbf{A^{(j)}_1}}  & \cdots & \frac{\partial u^{(n)}}{\partial \mathbf{A^{(j)}_m}}
\end{bmatrix}\begin{bmatrix}
\frac{\partial \mathbf{A^{(j)}_1}}{\partial u^{(j)}} \\ 
\vdots\\ \frac{\partial \mathbf{A^{(j)}_m}}{\partial u^{(j)}} 
\end{bmatrix} = \sum_{i:j \in Pa(u^{(i)})} \frac{\partial u^{(n)}}{\partial u^{(i)}} \frac{\partial u^{(i)}}{\partial u^{(j)}}$

## 6.5.5 Symbol-to-Symbol Derivatives

대수적 expression과 계산 그래프 모두 symbols 혹은 값이 정해지지 않은 variable 위에서 연산을 수행합니다. 이러한 대수적, 그래프 기반 representation을 symbolic representation이라고 부릅니다. 실제로 뉴럴네트워크를 학습할때 우리는 이러한 symbol에 특정한 값을 할당해야만 합니다. input에 해당하는 symbol에 특정한 수치적 값을 가지는 x로 대체하여 연산을 수행합니다.

계산 그래프와 input에 해당하는 값을 가지고 잇을때의 back-prop을 하는 몇몇 방법이 있습니다. <b>symbol-to-number</b> 접근 방법은 계산그래프가 주어지고 계산그래프의 input들이 주어지면 이 input에 해당하는 gradient들을 출력하는 방법입니다. 이들은 torch, caffe 등에서 사용됩니다.

<b>symbol-to-symbol</b> 방법은 계산 그래프가 주어지면 원하는 도함수에 대한 symbol을 그래프에 추가함으로써 실행시 gradient를 구하게 됩니다. 이런 방법은 theano, tensorflow에서 사용됩니다. 이러한 방법의 장점은 도함수가 원래의 expression과 같은 언어(그래프리컬)로 표현됩니다. 도함수는 단지 추가된 다른 계산 그래프기 때문에 back-prop을 여러번 실행 가능하며 2차 도함수도 쉽게 구할 수 있습니다. 

이 책에서는 symbol-to-symbol 방법을 사용하며 back-prop 알고리즘을 도함수를 위한 새로운 계산 그래프를 구성하는 것으로 설명합니다. 그래프의 부분 또한 특정 수치값을 input으로 넣어 평가 할 수 있습니다. 이는 언제 operation이 계산되어야 할지에 대한 명시를 피하게 해줍니다. 일반적인 graph evaluation engine은 모든 노드들을 그 부모의 값이 존재 할때 평가하도록 할 수 있습니다.

symbol-to-symbol 접근법에 대한 설명은 symbol-to-number 접근법에 대한 설명도 포함하고 있습니다. symbol-to-number 접근법은 symbol-to-symbol 에서 만들어진 그래프와 동일한 연산을 바로!! 수행하는 것으로 이해할 수 있씁니다. symbol-to-number 의 가장 큰 차이점은 back-prop을 위한 그래프를 명시적으로 노출하지 않습니다.

## 6.5.6 General Back-Propagation

back-prop 알고리즘은 매우 간단합니다. 스칼라 z와 그래프에서의 z의 부모중의 하나인 x에대한 gradient를 계산하기 위해선 z 그 자신의 gradient가 1인 것에서부터 시작합니다. $\frac{\mathrm{z} }{\mathrm{d} z} = 1$ 또한 z의 부모에 대한 gradient를 z를 출력한 어떤 함수에 대한 jacobian과 1과의 곱으로 계산할 수 있습니다. 이를 x에 도달할때 까지 계속 자코비안과 이전의 gradient를 곱해줍니다. z에서 부터 back-prop 중에 2개 이상의 path가 존재 한다면 간단하게 path 사이의 gradient를 다 더해줍니다.

더 정확하게 나타내면 그래프 G의 모든 노드들은 variable과 대응합니다. 가장 일반적인 형태를 얻기 위해서 variable을 일종의 tensor V로 설명합니다. Tensor는 어느 차원도 가질 수 있으며 tensor는 스칼라, 벡터, 매트릭스를 모두 포함합니다. 각각의 variable V가 다음과 같은 서브루틴과 관계가 있다고 가정합시다.

- get_operation(<b>V</b>) : 이 함수는 <b>V</b>를 계산하는 operation을 출력합니다. 이는 계산 그래프에서 V로 오는 edge로 표현됩니다. 예를들어 파이썬, c++ 에서는 매트릭스 곱에 관한 operation을 표현하는 클래스가 될 수 있습니다. 만약 C=AB 라는 매트릭스 곱으로 얻어지는 C라는 variable이 있다고 생각해 봅시다. 그렇다면 get_operation(C) 는 매트릭스 곱에 해당하는 포인터를 출력할 것입니다.

- get_consumers(<b>V</b>,G) : 계산 그래프 G에서 <b>V</b>의 자식에 해당하는 variable list를 출력합니다.

- get_inputs(<b>V</b>, G) : 계산 그래프 G에서의 <b>V</b>의 부모들에 해당하는 variable list를 출력합니다.

각각의 operation op는 bprop operation과 연관되어져 있습니다. 이 bprop operation은 gradient를 계산하기위한 계산그래프인 jacobian-vector product를 계산합니다. 이것이 back-prop 알고리즘의 가장 일반적인 설명입니다. 각 edge에 속해있는 operation은 어떻게 back-prop을 해야 할지에 대한 것을 알 고 있습니다. 예를 들어 C=AB 라는 매트릭스 곱 operation을 사용한다고 합시다. z스칼라 wrt C에 대한 gradient가 G에 주어졌다고 가정합시다. 매트릭스 곱 operation은 두개의 back-prop 규칙을 정의하는 역할을 하며 그중 하나는 backp-prop rule의 input argument입니다. 만약 bprop 메소드를 호출해 C wrt A의 gradient를 요구한다면 매트릭스 곱 operation의 bprop 메소드가 C wrt A gradient 가 $GB^T$ 라는 것을 알려주게 됩니다. 비슷하게 bprop 메소드를 호출하여 C wrt B gradient를 요구한다면 매트릭스 곱 operation은 bprop 메소드를 오해 이 gradient가 $A^TG$ 라는 것을 알려줘야 합니다. back-prop 알고리즘은 그 자체로 어떠한 미분에 대한 규칙을 알지 못합니다. 이들은 오직 각 operation의 bprop 메소드가 올바른 행동을 할것을 요구할 뿐입니다. 즉 모든 operation에 대한 bprop 메소드는 다음과 같이 출력해야 합니다.

## $op.bprop(inputs, X,G) = \sum_i(\bigtriangledown _\mathbf{X} op.f(inputs)_i)G_i$

이는 위의 6.47에 표현된 체인룰의 구현입니다. 여기서 inputs이란 operation을 수행하기 위해 필요한 inputs list이며 op.f 는 해당하는 operation의 구현입니다. X는 graidnet를 구하고 싶어 하는 input이며, G는 operation output에 대한 gradient가 됩니다.(이전 레이어에서 받아온 gradient를 말하나?) 

op.prop 메소드는 모든 inputs list들이 서로 다르다고 가정해야 합니다. 예를 들어 mul operator가 mul(x,x) =x^2 을 수행했다고 생각해 봅시다. op.bprop 메소드는 여전히 lhs, rhs 에 대해 모두 x라는 도함수를 출력해야 합니다. back-prop 알고리즘은 나중에 이 둘을 더해 2x라는 값을 얻게 될것이고 이는 올바른 미분이 됩니다.

back-prop에 대한 소프트웨어 들은 대부분 operation, operation.bprop 메소드를 제공합니다. 그래서 딥러닝 라이브러리의 유저들은 그래프를 통해 back-prop을 수행할 수 있습니다. back-prop에 대한 새로운 구현을 하려는 엔지니어들 혹은 새로운 opeartion을 추가하려는 고급 유저들은 새로운 operation에 대해 수동으로 op.bprop을 정의해줘야 합니다.

섹션 6.5.2에서 back-prop 알고리즘이 chain-rule에서 중복되어 나타나는 sub-experssion의 계산을 피하기 위해 설계되었다고 설명했습니다. naive한 구현은 이러한 반복되는 sub-experssion 때문에 지수적 실행시간을 가질 수 있습니다. 지금까지 back-prop 알고리즘에 대해 명시를 하였고 이제 이 알고리즘의 연산량에 대해 이해할 수 있씁니다. 만약 우리가 각 opearation 을 평가하는 것이 같은 cost라고 가정한다면 back-prop의 연산량은 operation의 수라고 할 수 있습니다. 하지만 하나의 operation은 매우 많은 대수적 연산으로 이루어져 있을 수 있습니다. (예를들어 매트릭스 곱을 하나의 operation으로 삼을 수 있습니다.) n개의 노드로 이루어진 그래프에서 gradient를 계산하는 것은 O(n^2) 를 넘을 수 없으며 memory 도 O(n^2)를 넘을 수 없습니다. 여기서 계산 그래프에서의 operation의 갯수가 n이며 각 operation의 실행시간은 매우 차이가 난다는 것이 중요합니다. 예를 들어 만개의 요소들을 가진 매트릭스 곱을 그래프에서의 하나의 opeartion으로 삼을 수 있습니다 이 때 gradient를 계산하는 것은 대부분 O(n^2) 을 필요로 하며 이는 forward propagtation에서의 worst case가 모든 노드가 연결되어져 있는 것이기 때문입니다. back-prop 알고리즘은 원래 그래프의 모든 edge마다 jacobian-vector propduct node를 추가하며 이 노드는 O(1) 을 가집니다. 실제로 많이 사용되는 그래프의 경우에는 상황이 좀더 낫습니다. 대부분의 cost function을 사용하는 뉴럴네트워크는 성긴 형태로 엮겨 있고 이는 back-prop이 O(n) cost를 가지도록 합니다. naive 한 접근방법에서는 exponentially many node를 실행해야 할 수 있습니다. 이러한 잠재적인 cost는 6.49의 재귀적인 chain rule을 재귀적이지 않게 작성함으로써 볼 수 있습니다.

노드 j에서 노드 n까지의 경로가 기하급수적으로 증가할 수 있기 때문에 summation안의 수식의 숫자가 forward 그래프의 깊이에 지수적으로 증가 할 수 있습니다. 이러한 cost는 sub-expression에 대한 미분의 계산을 여러번 하기 떄문에 발생합니다. 이러한 것을 피하기 위해 back-prop 알고리즘을 일종의 table-filling 알고리즘으로 생각할 수 있습니다. 그래프내의 각각 노드들은 그 노드에 gradient에 해당하는 table-slot을 가지며 이러한 테이블을 채움으로써 back-prop 은 subexpression을 여러번 계산하는 것을 회피합니다. 이러한 table-filling 전략은 종종 dynamic programming이라고 불립니다.

### back-prop

- subexpression에 대한 gradient의 반복적인 계산을 줄여 연산량을 줄임, 하지만 메모리는 더 먹는다.
- symbol-to-symbol은 도함수에 대한 node를 그래프에 추가하는 것이며 symbol-to-number는 일종의 table-filling 알고리즘이라고 생각 할 수 있다.



## 6.5.8 Complications

- 위에서는 여러개의 output에 대한 operation은 다루지 않았다. 많은 소프트웨어에서느 이를 지원하는데 예를 들어 maximum value, index를 계산하고 싶다면 이를 2개의 opeartion으로 구현하는 것보다 two output operation으로 구현하면 메모리를 줄일 수 있다.
- back-prop은 종종 tensor의 summation이 많이 포함된다. gradient를 다 구하고 그 다음 summation 하는 것보다 single buffer를 유지하면서 add를 하는 것이 메모리 효율이 좋다!



## 6.6 Historical Notes

피드포워드 네트워크의 성능을 향상시킨 중요한 변화중 하나는 sigmoid unit대신 relu와 같은 piecewise linear hidden unit을 사용한 것입니다. max(0,z) 와 같은 recitication는 1970년대의 초창기 뉴럴네트워크 연구에서 도입되었습니다. 이러한 초창기 모델은 z를 linear function이 아닌 non-linear function을 사용했습니다. 이렇게 초창기부터 rectification이 사용되어졌지만 1980년대에는 rectification이 대부분 sigmoid 로 교체되었습니다. 아마 이는 sigmoid가 작은 뉴럴네트워크에서 더 잘 작동했기 때문일 것입니다. 2000년대 초반은 relu의 미분 불가능한 점때문에 사용을 꺼려 하였습니다. 이는 2009년대 에 가서야 바뀌게 되었는데 jarrett는 rectification non-linearity가 recognition system의 성능향상중 가장 큰 요인이라고 주장하였습니다.

jarrett는 작은 데이터셋에서 recitification non-linearlity를 사용하는 것이 심지어 히든레이어의 가중치를 학습하는 것보다 더 중요하다는 것을 발견했습니다. 가중치를 랜덤하게 주고 relu를 사용하는 것만으로도 정보를 propagation할 수 있고 classfication도 가능하다는 것을 보였습니다.

Glorot 은 relu를 사용한 딥한 네트워크가 sigmoid, tanh 같은 two-sided satuation 을 사용하는 것보다 좋다는 것을 보였습니다.

relu 는 또한 신경과학이 계속해서 딥러닝 알고리즘에 영향을 주고 있다는 것을 보여주는 재밌는 예입니다. Glorot는 생물학적 조건에서부터 relu에 대한 영감을 얻었습니다. half-rectifying non-linearlity는 다음과 같은 뉴런의 특징을 위해 설계되어졌습니다.

- 1. 어떤 입력에서는 생물학적 뉴런이 완전히 활성화하지 않는다
- 2. 어떤 입력에서는 생물학적 뉴런의 출력이 입력에 비례한다.
- 3. 대부분은 시간동안 생물학적 뉴런은 활성화 하지 않는다. (즉 뉴런은 sparse activation의 특징을 가진다)

